In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import scipy
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go

## 1. Load microarray result

In [ ]:
pf=pd.read_excel("Clean_results_2_2019.xlsx")

## 2. Load custom feature scoring and feature extraction

### 2.1. Use this for the composition-based feature scoring

In [ ]:
#Load feature scoring file
df=pd.read_excel("aa_properties2_Cpos_19cat.xlsx") #This one count atom types in each amino acid

In [ ]:
#Feature extraction
header=np.asarray(list(df.columns.values))
feat=np.delete(header, 0, axis=None)

In [ ]:
#Count each feature for each peptide in the microarray result
#Apply the function in the peptide sequence
def aa_count(peptide):
#Iterate through each amino acid in peptide sequence
    rows=[]
    for aa in peptide:
        rows.append(df[df.aa==aa])
    return pd.concat(rows).sum()

In [ ]:
#Create a new df with the function applied to Peptide column from pf
transpf=pf['Peptide'].apply(aa_count)
transpf

In [ ]:
#Merge transpf table containing amino acid sums to original raw data table pf
mergepf = pd.concat([pf, transpf], axis=1, sort=False)
mergepf["Prot_pep"] = mergepf["Protein"].map(str) +"_"+ mergepf["Peptide"]
mergepf["Prot_res_pep"] = mergepf["Protein"].map(str) +"_"+ mergepf["Residue number"].map(str)+"_"+ mergepf["Peptide"]
mergepf

In [ ]:
features=[mergepf.filter(feat)]
featx=np.reshape(features, (5395, 19), order='A')

### 2.2. Use this for the Seq and UniSeq-based feature scoring

In [ ]:
#Seq
#aadict={'A': 0.78, 'C': 3.14, 'D': 1.25, 'E': 0.94, 'F': 1.07, 'G': 1.13, 'H': 1.03, 'I': 1.26, 'K': 0.85, 'L': 0.91,
#        'M': 0.41, 'N': 1.32, 'P': 1.73, 'Q': 0.93, 'R': 1.75, 'S': 1.31, 'T': 1.57, 'Y': 1.31, 'V': 1.11, 'W': 0.98}

#UniSeq
aadict={'A': 10, 'C': 20, 'D': 30, 'E': 40, 'F': 50, 'G': 60, 'H': 70, 'I': 80, 'K': 90, 'L': 100,
        'M': 110, 'N': 120, 'P': 130, 'Q': 140, 'R': 150, 'S': 160, 'T': 170, 'Y': 180, 'V': 190, 'W': 200}

In [ ]:
## Replace each amino acid with custom scoring in the microarray result

In [ ]:
def aa_replace(peptide):
    rows=[]
    for aa in peptide:
        #pepsplit=peptide.split()
        rows.append(aadict[aa])
    return rows

In [ ]:
transpfseq=pf['Peptide'].apply(aa_replace)

In [ ]:
#Create dataframe with the replaced values
zfs=[]
for idx,row in transpfseq.iteritems():
    zf=pd.DataFrame(row)
    zfs.append(zf.T)
aaconv=pd.concat(zfs)
aaconv=aaconv.reset_index()
aaconv['Peptide']=pf['Peptide']

In [ ]:
#Merge with original dataframe
mergepf = pd.concat([pf,aaconv], axis=1, sort=False)

In [ ]:
#Fourier transform of the replaced values
from scipy.fft import fft
import matplotlib.pylab as plt
fftdf=pd.DataFrame(columns = ["1","2","3","4","5","6"])
for idx,row in aaconv.iterrows():
    fftlst=fft(np.array(row[list(range(0,15))]))
    plt.plot(np.abs(2/15*fftlst[1:15//2])) #need to correct
    new_df = pd.DataFrame(data = [np.abs(2/15*fftlst[1:15//2])], columns = ["1","2","3","4","5","6"])
    fftdf= pd.concat([fftdf, new_df])

fftdf=fftdf.reset_index()
newfft= pd.concat([pf,fftdf], axis=1, sort=False)

In [ ]:
features=[u"1",u"2",u"3",u"4",u"5",u"6"]
featx = np.array(newfft[features])

## 3. Prediction

### 3.1. Use this for the composition-based feature scoring

In [ ]:
hls=(1,) #hidden layer shape
#The optimized grid search hyperparams
clf = MLPClassifier(solver='adam', random_state=1, max_iter=10000, hidden_layer_sizes=hls)

def pept_class(features):
    X = StandardScaler().fit_transform(featx)
    #X_train, X_test, y_train, y_test = train_test_split(
    #    X, np.array(mergepf[u'Survivin, 1 µg/ml']>0), test_size=0.5, shuffle=True)
    #clf.fit(X_train, y_train)
    #predicted=clf.predict(X_test)
    #return metrics.confusion_matrix(y_test, predicted)
    clf.fit(X, np.array(mergepf[u'Survivin, 1 µg/ml']>0))
    predicted=clf.predict(X)
    return metrics.confusion_matrix(np.array(mergepf[u'Survivin, 1 µg/ml']>0), predicted)
confmtxs=[]
for i in range(0,100):
    confmtx=pept_class(features)
    confmtxs.append(confmtx)
confmtxs

### 3.2. Use this for the Seq and UniSeq-based feature scoring

In [ ]:
hls=(1,) #hidden layer shape
clf = MLPClassifier(solver='adam',random_state=1, max_iter=10000, hidden_layer_sizes=hls)

features=[u'1',u'2',u'3',u'4',u'5',u'6']

def pept_class(features):
    X = np.array(newfft[features])
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X, np.array(newfft[u'Survivin, 1 µg/ml']>0), test_size=0.5, shuffle=True)
    clf.fit(X_train, y_train)
    predicted=clf.predict(X_train)
    return metrics.confusion_matrix(y_train, predicted)
confmtxs=[]
for i in range(0,100):
    confmtx=pept_class(features)
    confmtxs.append(confmtx)
confmtxs

In [ ]:
stat=mean.tolist()+se.tolist()
stat.insert(0,hls[0])

In [ ]:
stat1=pd.DataFrame(stat)
stat1

In [ ]:
stat1[1]=stat
stat1

In [ ]:
stats = stat1.transpose()
stats.columns=['Nodes','Mean_Pprec','Mean_Nprec','Mean_Prec','Mean_Nrec','Mean_Acc','Mean_PF1',
                                      'Mean_NF1','SE_Pprec','SE_Nprec','SE_Prec','SE_Nrec','SE_Acc','SE_PF1',
                                      'SE_NF1']
stats.to_excel('output.xlsx', index=True, header=True)

## 4. [FIG 2] Different features

In [ ]:
#Load statistics file
pf=pd.read_excel("output.xlsx")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    name='Mean Accuracy',
    x=pf['Nodes'], y=pf['Mean_Acc'],
    error_y=dict(type='data', array=pf['SE_Acc'])
    ,mode='markers'
    #,mode='markers+lines'
))
fig.add_trace(go.Scatter(
    name='Mean F1 Positive',
    x=pf['Nodes'], y=pf['Mean_PF1'],
    error_y=dict(type='data', array=pf['SE_PF1'])
    ,mode='markers'
    #,mode='markers+lines'
))
fig.add_trace(go.Scatter(
    name='Mean F1 Negative',
    x=pf['Nodes'], y=pf['Mean_NF1'],
    error_y=dict(type='data', array=pf['SE_NF1'])
    ,mode='markers'
    #,mode='markers+lines'
))
fig.update_traces(marker_size=8,line=dict(width=2))
fig.update_layout(
    height=800,width=1000,
    #barmode='group',
    #title="1 Hidden Layer of Varying Number of Nodes",
    xaxis_title="Features",
    #yaxis_title="Mean Accuracy (%)",
    #legend_title="Legend",
    font=dict(
        family="Open Sans, verdana, arial, sans-serif",
        size=25,
        color="Rebecca Purple"
    )
)

In [ ]:
fig.write_image("fig_diff_feat_2.svg")

## 5. [FIG 3] Different number of nodes

In [ ]:
#Load statistics file
val=pd.read_excel("output_val.xlsx")
noval=pd.read_excel("output_noval.xlsx")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    name='Mean F1 Negative V',
    x=val['Nodes'], y=val['Mean_NF1'],
    error_y=dict(type='data', array=val['SE_NF1'])
    ,mode='markers+lines'
    ,line_color='teal'
))
fig.add_trace(go.Scatter(
    name='Mean F1 Negative NV',
    x=val['Nodes'], y=noval['Mean_NF1'],
    error_y=dict(type='data', array=noval['SE_NF1'])
    ,mode='markers+lines'
    ,line_color='lightseagreen'
    ,fill='tonexty'
))

fig.add_trace(go.Scatter(
    name='Mean F1 Positive V',
    x=val['Nodes'], y=val['Mean_PF1'],
    error_y=dict(type='data', array=val['SE_PF1'])
    ,mode='markers+lines'
    ,line_color='deeppink'
))
fig.add_trace(go.Scatter(
    name='Mean F1 Positive NV',
    x=val['Nodes'], y=noval['Mean_PF1'],
    error_y=dict(type='data', array=noval['SE_PF1'])
    ,mode='markers+lines'
    ,line_color='hotpink'
    ,fill='tonexty'
))

fig.add_trace(go.Scatter(
    name='Mean Accuracy V',
    x=val['Nodes'], y=val['Mean_Acc'],
    error_y=dict(type='data', array=val['SE_Acc'])
    ,mode='markers+lines'
    ,line_color='royalblue'
))
fig.add_trace(go.Scatter(
    name='Mean Accuracy NV',
    x=val['Nodes'], y=noval['Mean_Acc'],
    error_y=dict(type='data', array=noval['SE_Acc'])
    ,mode='markers+lines'
    ,line_color='dodgerblue'
    ,fill='tonexty'
))

fig.update_traces(marker_size=8,line=dict(width=2))
fig.update_layout(
    height=800,width=1000,
    xaxis_title="Number of Nodes",
    yaxis_title="Performance Metrics (%)",
    font=dict(
        family="Open Sans, verdana, arial, sans-serif",
        size=25,
        color="Rebecca Purple"
    )
)
fig.show()

In [ ]:
fig.write_image("fig_valnoval.svg")

## 6. [FIG 4] Visualize NN

In [ ]:
training_set_outputs = np.array([y_train]).T
training_set_outputs.shape[1]

In [ ]:
from matplotlib import pyplot
from math import cos, sin, atan
from palettable.matplotlib import Plasma_3
from time import localtime, strftime

class Neuron():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def draw(self, neuron_radius, id=-1):
        circle = pyplot.Circle((self.x, self.y), radius=neuron_radius, fill=False)
        pyplot.gca().add_patch(circle)
        pyplot.gca().text(self.x, self.y-0.15, str(id), size=20, ha='center')

class Layer():
    def __init__(self, network, number_of_neurons, number_of_neurons_in_widest_layer):
        self.vertical_distance_between_layers = 20
        self.horizontal_distance_between_neurons = 6
        self.neuron_radius = 2.8
        self.number_of_neurons_in_widest_layer = number_of_neurons_in_widest_layer
        self.previous_layer = self.__get_previous_layer(network)
        self.y = self.__calculate_layer_y_position()
        self.neurons = self.__intialise_neurons(number_of_neurons)

    def __intialise_neurons(self, number_of_neurons):
        neurons = []
        x = self.__calculate_left_margin_so_layer_is_centered(number_of_neurons)
        for iteration in range(number_of_neurons):
            neuron = Neuron(x, self.y)
            neurons.append(neuron)
            x += self.horizontal_distance_between_neurons
        return neurons

    def __calculate_left_margin_so_layer_is_centered(self, number_of_neurons):
        return self.horizontal_distance_between_neurons * (self.number_of_neurons_in_widest_layer - number_of_neurons) / 2

    def __calculate_layer_y_position(self):
        if self.previous_layer:
            return self.previous_layer.y + self.vertical_distance_between_layers
        else:
            return 0

    def __get_previous_layer(self, network):
        if len(network.layers) > 0:
            return network.layers[-1]
        else:
            return None

    def __line_between_two_neurons(self, neuron1, neuron2, weight=0.4, textoverlaphandler=None):
        angle = atan((neuron2.x - neuron1.x) / float(neuron2.y - neuron1.y))
        x_adjustment = self.neuron_radius * sin(angle)
        y_adjustment = self.neuron_radius * cos(angle)

        # assign colors to lines depending on the sign of the weight
        #color=Tableau_10.mpl_colors[0]
        #if weight > 0: color=Tableau_10.mpl_colors[1]
        color=Plasma_3.mpl_colors[0]
        if weight > 0: color=Plasma_3.mpl_colors[1]

        # assign different linewidths to lines depending on the size of the weight
        abs_weight = abs(weight)        
        if abs_weight > 0.5: 
            linewidth = 10*abs_weight
        elif abs_weight > 0.8: 
            linewidth =  100*abs_weight
        else:
            linewidth = abs_weight

        # draw the weights and adjust the labels of weights to avoid overlapping
        if abs_weight > 0.5: 
            # while loop to determine the optimal locaton for text lables to avoid overlapping
            index_step = 2
            num_segments = 10   
            txt_x_pos = neuron1.x - x_adjustment+index_step*(neuron2.x-neuron1.x+2*x_adjustment)/num_segments
            txt_y_pos = neuron1.y - y_adjustment+index_step*(neuron2.y-neuron1.y+2*y_adjustment)/num_segments
            while ((not textoverlaphandler.getspace([txt_x_pos-0.5, txt_y_pos-0.5, txt_x_pos+0.5, txt_y_pos+0.5])) and index_step < num_segments):
                index_step = index_step + 1
                txt_x_pos = neuron1.x - x_adjustment+index_step*(neuron2.x-neuron1.x+2*x_adjustment)/num_segments
                txt_y_pos = neuron1.y - y_adjustment+index_step*(neuron2.y-neuron1.y+2*y_adjustment)/num_segments

            # print("Label positions: ", "{:.2f}".format(txt_x_pos), "{:.2f}".format(txt_y_pos), "{:3.2f}".format(weight))
            a=pyplot.gca().text(txt_x_pos, txt_y_pos, "{:3.2f}".format(weight), size=20, ha='center')
            a.set_bbox(dict(facecolor='white', alpha=0.5))
            # print(a.get_bbox_patch().get_height())

        line = pyplot.Line2D((neuron1.x - x_adjustment, neuron2.x + x_adjustment), (neuron1.y - y_adjustment, neuron2.y + y_adjustment), linewidth=linewidth, color=color)
        pyplot.gca().add_line(line)

    def draw(self, layerType=0, weights=None, textoverlaphandler=None):
        j=0 # index for neurons in this layer
        feat=['0','Pro-MC', 'Carboxyl', 'Amide', 'His', 'Trp', 'Phe-Tyr', 'CH2','CH', 'CH3', 'OH', 'SH', 'S', 'NH3', 
              'Arg', 'MC'] #feature labels
        for neuron in self.neurons:            
            i=0 # index for neurons in previous layer
            if layerType == 0:
                neuron.draw(self.neuron_radius, id=feat[j+1])
            else:
                neuron.draw( self.neuron_radius, id=j+1 )
            if self.previous_layer:
                for previous_layer_neuron in self.previous_layer.neurons:
                    self.__line_between_two_neurons(neuron, previous_layer_neuron, weights[i,j], textoverlaphandler)
                    i=i+1
            j=j+1
        
        # write Text
        x_text = self.number_of_neurons_in_widest_layer * self.horizontal_distance_between_neurons
        if layerType == 0:
            pyplot.text(x_text, self.y, 'Input Layer', fontsize = 20)
        elif layerType == -1:
            pyplot.text(x_text, self.y, 'Output Layer', fontsize = 20)
        else:
            pyplot.text(x_text, self.y, 'Hidden Layer '+str(layerType), fontsize = 20)

# A class to handle Text Overlapping
# The idea is to first create a grid space, if a grid is already occupied, then
# the grid is not available for text labels.
class TextOverlappingHandler():
    # initialize the class with the width and height of the plot area
    def __init__(self, width, height, grid_size=0.2):
        self.grid_size = grid_size
        self.cells = np.ones((int(np.ceil(width / grid_size)), int(np.ceil(height / grid_size))), dtype=bool)

    # input test_coordinates(bottom left and top right), 
    # getspace will tell you whether a text label can be put in the test coordinates
    def getspace(self, test_coordinates):
        x_left_pos = int(np.floor(test_coordinates[0]/self.grid_size))
        y_botttom_pos = int(np.floor(test_coordinates[1]/self.grid_size))
        x_right_pos = int(np.floor(test_coordinates[2]/self.grid_size))
        y_top_pos = int(np.floor(test_coordinates[3]/self.grid_size))
        if self.cells[x_left_pos, y_botttom_pos] and self.cells[x_left_pos, y_top_pos] \
        and self.cells[x_right_pos, y_top_pos] and self.cells[x_right_pos, y_botttom_pos]:
            for i in range(x_left_pos, x_right_pos):
                for j in range(y_botttom_pos, y_top_pos):
                    self.cells[i, j] = False

            return True
        else:
            return False

class NeuralNetwork():
    def __init__(self, number_of_neurons_in_widest_layer):
        self.number_of_neurons_in_widest_layer = number_of_neurons_in_widest_layer
        self.layers = []
        self.layertype = 0

    def add_layer(self, number_of_neurons ):
        layer = Layer(self, number_of_neurons, self.number_of_neurons_in_widest_layer)
        self.layers.append(layer)

    def draw(self, weights_list=None):
        # vertical_distance_between_layers and horizontal_distance_between_neurons are the same with the variables of the same name in layer class
        vertical_distance_between_layers = 20
        horizontal_distance_between_neurons = 6
        overlaphandler = TextOverlappingHandler(\
            self.number_of_neurons_in_widest_layer*horizontal_distance_between_neurons,\
            len(self.layers)*vertical_distance_between_layers, grid_size=0.2 )

        pyplot.figure(figsize=(30, 30))
        for i in range( len(self.layers) ):
            layer = self.layers[i]                                
            if i == 0:
                layer.draw( layerType=0 )
            elif i == len(self.layers)-1:
                layer.draw( layerType=-1, weights=weights_list[i-1], textoverlaphandler=overlaphandler)
            else:
                layer.draw( layerType=i, weights=weights_list[i-1], textoverlaphandler=overlaphandler)

        pyplot.axis('scaled')
        pyplot.axis('off')
        #pyplot.title( 'Neural Network architecture', fontsize=15 )
        figureName='ANN_'+strftime("%Y%m%d_%H%M%S", localtime())+'.svg'
        pyplot.savefig(figureName, dpi=300, bbox_inches="tight")
        pyplot.show()

class DrawNN():
    # para: neural_network is an array of the number of neurons 
    # from input layer to output layer, e.g., a neural network of 5 nerons in the input layer, 
    # 10 neurons in the hidden layer 1 and 1 neuron in the output layer is [5, 10, 1]
    # para: weights_list (optional) is the output weights list of a neural network which can be obtained via classifier.coefs_
    def __init__( self, neural_network, weights_list=None ):
        self.neural_network = neural_network
        self.weights_list = weights_list
        # if weights_list is none, then create a uniform list to fill the weights_list
        if weights_list is None:
            weights_list=[]
            for first, second in zip(neural_network, neural_network[1:]):
                tempArr = np.ones((first, second))*0.4
                weights_list.append(tempArr)
            self.weights_list = weights_list
        
    def draw( self ):
        widest_layer = max( self.neural_network )
        network = NeuralNetwork( widest_layer )
        for l in self.neural_network:
            network.add_layer(l)
        network.draw(self.weights_list)

In [ ]:
#classifier.fit(X, y.ravel())
network_structure = np.hstack(([X_train.shape[1]], np.asarray(clf.hidden_layer_sizes), training_set_outputs.shape[1]))

# Draw the Neural Network with weights
network=DrawNN(network_structure, clf.coefs_)
fig=network.draw()

## 7. [FIG S1, S2, S3] T-SNE FIGURES

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(featx)

In [ ]:
vf = pd.DataFrame()
vf["y"] = np.array(mergepf[u'Survivin, 1 µg/ml']!=0)
vf["comp-1"] = z[:,0]
vf["comp-2"] = z[:,1]
vff=pd.concat([vf,transpf], axis=1, sort=False)
vff['Uniprot ID']=pf['Uniprot ID']
vff['Survivin, 1 µg/ml']=pf['Survivin, 1 µg/ml']

In [ ]:
import plotly.express as px
#Use this to see binders vs non-binders
fig = px.scatter(x=vff["comp-1"], y=vff["comp-2"],color=vff.y.tolist(), color_discrete_sequence=["cyan", "lightcoral"])

#Use this to see binding intensity
#fig = px.scatter(vff, x="comp-1", y="comp-2",color=vff['Survivin, 1 µg/ml'].tolist(), 
#                 color_continuous_scale=px.colors.sequential.Inferno_r, hover_data=['Uniprot ID', 'Survivin, 1 µg/ml'])

fig.update_traces(marker=dict(size=5,
                              line=dict(width=0.5,
                                        color='Snow')),
                  selector=dict(mode='markers'))
fig.show()